In [1]:
#data at https://moneypuck.com/data.htm
import pandas as pd
import os
import json

In [2]:
skater_drop_features = ['I_F_penalityMinutes',
 'penalityMinutes',
 'penalties',
 'I_F_playContinuedInZone',
 'I_F_playContinuedOutsideZone',
 'OnIce_A_flurryScoreVenueAdjustedxGoals',
 'OnIce_A_scoreVenueAdjustedxGoals',
 'OffIce_F_shotAttempts',
 'I_F_flyShiftEnds',
 'OnIce_F_flurryScoreVenueAdjustedxGoals',
 'OnIce_A_shotAttempts',
 'OnIce_A_missedShots',
 'OffIce_A_shotAttempts',
 'OnIce_A_scoreAdjustedUnblockedShotAttempts',
 'OnIce_A_scoreAdjustedShotsAttempts',
 'OnIce_A_lowDangerGoals',
 'OnIce_A_xGoalsFromActualReboundsOfShots',
 'I_F_flyShiftStarts',
 'I_F_dZoneShiftStarts',
 'I_F_dZoneShiftEnds',
 'OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted',
 'OnIce_A_xGoals_with_earned_rebounds',
 'I_F_neutralZoneShiftStarts',
 'OnIce_F_flurryAdjustedxGoals',
 'OnIce_A_unblockedShotAttempts',
 'OnIce_A_xGoals',
 'OnIce_A_highDangerxGoals',
 'OnIce_A_flurryAdjustedxGoals',
 'OnIce_A_lowDangerxGoals',
 'OnIce_A_mediumDangerxGoals',
 'OnIce_A_highDangerGoals',
 'OnIce_A_mediumDangerShots',
 'OnIce_A_highDangerShots',
 'OnIce_A_lowDangerShots',
 'OnIce_A_goals',
 'OnIce_A_blockedShotAttempts',
 'OnIce_A_reboundGoals',
 'OnIce_A_rebounds',
 'I_F_oZoneShiftEnds']

In [6]:
folder_path = './'
csv_folder = os.path.join(folder_path, 'csvs')
output_folder = os.path.join(folder_path, 'jsons')

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Loop through all CSV files in the directory
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        df = pd.read_csv(file_path)

        # Convert to list-of-dict format
        data_as_json = df.to_dict(orient='records')

        # Define output filename (.txt with same name as .csv)
        base_name = os.path.splitext(filename)[0].replace(' ', '_')
        output_file = os.path.join(output_folder, f"{base_name}.txt")

        # Save JSON to .txt
        with open(output_file, 'w') as f:
            json.dump(data_as_json, f, indent=4)

        print(f"Saved {output_file}")

Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/skaters_(9).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/goalies.txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/goalies_(4).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/lines_(5).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/skaters_(15).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/skaters_(5).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/teams_(2).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/goalies_(14).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/lines_(13).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/lines_(9).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/da

In [ ]:
folder_path = './'
csv_folder = os.path.join(folder_path, 'csvs')
output_folder = os.path.join(folder_path, 'texts')

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert each row into a readable paragraph
# def row_to_text(row):
#     parts = [f"{col}: {row[col]}" for col in df.columns]
#     return ", ".join(parts)

# # # # Combine all rows into one big string
# full_text = '\n\n'.join(df.apply(row_to_text, axis=1))

# # Save to text file
# with open('nhl_stats_full.txt', 'w') as f:
#     f.write(full_text)

# Loop through all CSV files in the directory
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        df = pd.read_csv(file_path)
#         print(filename)
        if "skater" in filename:
            df = df.drop(columns=skater_drop_features)
            
        def row_to_text(row):
            return ", ".join(f"{col}: {row[col]}" for col in df.columns)

        # Convert to list-of-dict format
        full_text = '\n\n'.join(df.apply(row_to_text, axis=1))

        base_name = os.path.splitext(filename)[0].replace(' ', '_')
        output_file = os.path.join(output_folder, f"{base_name}.txt")
        with open(output_file, 'w') as f:
            f.write(full_text)

        print(f"Saved {output_file}")


Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts2/skaters_(9).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts2/goalies.txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts2/goalies_(4).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts2/lines_(5).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts2/skaters_(15).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts2/skaters_(5).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts2/teams_(2).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts2/goalies_(14).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts2/lines_(13).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts2/lines_(9).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_

In [4]:
folder_path = './'
csv_folder = os.path.join(folder_path, 'csvs_john')
output_folder = os.path.join(folder_path, 'texts_john_limit500000')

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

MAX_FILE_SIZE = 500000

for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        df = pd.read_csv(file_path)

#         if "skater" in filename:
#             df = df.drop(columns=skater_drop_features)

        def row_to_text(row):
            return ", ".join(f"{col}: {row[col]}" for col in df.columns)

        base_name = os.path.splitext(filename)[0].replace(' ', '_')

        parts = []
        current_part = []
        current_size = 0

        for row_text in df.apply(row_to_text, axis=1):
            row_size = len(row_text.encode('utf-8')) + 2  # +2 for newlines
            if current_size + row_size > MAX_FILE_SIZE:
                parts.append(current_part)
                current_part = []
                current_size = 0
            current_part.append(row_text)
            current_size += row_size

        if current_part:
            parts.append(current_part)

        for i, part in enumerate(parts):
            suffix = f"_part{i+1}" if len(parts) > 1 else ""
            output_file = os.path.join(output_folder, f"{base_name}{suffix}.txt")
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write('\n\n'.join(part))
            print(f"Saved {output_file} (Size: {os.path.getsize(output_file)} bytes)")

Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2010)_part1.txt (Size: 497842 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2010)_part2.txt (Size: 498210 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2010)_part3.txt (Size: 498006 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2010)_part4.txt (Size: 497162 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2010)_part5.txt (Size: 498130 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2010)_part6.txt (Size: 497838 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2010)_part7.txt (Size: 497549 bytes)
Saved /Users/carlg/Desktop/

Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2012)_part1.txt (Size: 497601 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2012)_part2.txt (Size: 497515 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2012)_part3.txt (Size: 496685 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2012)_part4.txt (Size: 497131 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2012)_part5.txt (Size: 497376 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2012)_part6.txt (Size: 497245 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2012)_part7.txt (Size: 497875 bytes)
Saved /Users/

Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2009)_part1.txt (Size: 495754 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2009)_part2.txt (Size: 499745 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2009)_part3.txt (Size: 499394 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2009)_part4.txt (Size: 496932 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2009)_part5.txt (Size: 498614 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2009)_part6.txt (Size: 495682 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2009)_part7.txt (Size: 498153 bytes)
Saved /Users/

Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2011)_part1.txt (Size: 497718 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2011)_part2.txt (Size: 497739 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2011)_part3.txt (Size: 497720 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2011)_part4.txt (Size: 497195 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2011)_part5.txt (Size: 498168 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2011)_part6.txt (Size: 497630 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2011)_part7.txt (Size: 497678 bytes)
Saved /Users/carlg/Desktop/

Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2014)_part1.txt (Size: 499890 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2014)_part2.txt (Size: 499463 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2014)_part3.txt (Size: 495749 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2014)_part4.txt (Size: 495638 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2014)_part5.txt (Size: 498275 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2014)_part6.txt (Size: 497171 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2014)_part7.txt (Size: 497409 bytes)
Saved /Users/

...

Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2010)_part1.txt (Size: 499328 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2010)_part2.txt (Size: 498540 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2010)_part3.txt (Size: 499113 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2010)_part4.txt (Size: 499284 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2010)_part5.txt (Size: 499005 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2010)_part6.txt (Size: 499979 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2010)_part7.txt (Size: 497084 bytes)
Saved /Users/

Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2008)_part1.txt (Size: 498004 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2008)_part2.txt (Size: 498263 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2008)_part3.txt (Size: 497948 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2008)_part4.txt (Size: 497953 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2008)_part5.txt (Size: 497562 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2008)_part6.txt (Size: 497498 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2008)_part7.txt (Size: 497371 bytes)
Saved /Users/carlg/Desktop/

Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/teams_(2008).txt (Size: 461488 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2009)_part1.txt (Size: 497181 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2009)_part2.txt (Size: 497841 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2009)_part3.txt (Size: 497549 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2009)_part4.txt (Size: 498083 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2009)_part5.txt (Size: 497932 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2009)_part6.txt (Size: 498119 bytes)
Saved /Users/carlg/Desktop/Code/S

Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2013)_part1.txt (Size: 497972 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2013)_part2.txt (Size: 497289 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2013)_part3.txt (Size: 497926 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2013)_part4.txt (Size: 498004 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2013)_part5.txt (Size: 498056 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2013)_part6.txt (Size: 497998 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/lines_(2013)_part7.txt (Size: 498065 bytes)
Saved /Users/carlg/Desktop/

Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2011)_part1.txt (Size: 495821 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2011)_part2.txt (Size: 498794 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2011)_part3.txt (Size: 497165 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2011)_part4.txt (Size: 498690 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2011)_part5.txt (Size: 499602 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2011)_part6.txt (Size: 497464 bytes)
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_john_limit500000/skaters_(2011)_part7.txt (Size: 497129 bytes)
Saved /Users/

In [ ]:
folder_path = '/Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data'
csv_folder = os.path.join(folder_path, 'csvs')
output_folder = os.path.join(folder_path, 'texts_chunked')

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

MAX_CHARS = 1000  # Max characters per chunk

# Loop through all CSV files in the directory
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        df = pd.read_csv(file_path)

        # Convert a row into a readable format with ability to slice
        def row_to_text_segments(row, max_len):
            name = f"name: {row['name']}" if 'name' in row else ""
            season = f"season: {row['season']}" if 'season' in row else ""
            context = f"{name}, {season}"
            other_parts = [f"{col}: {row[col]}" for col in df.columns if col not in ['name', 'season']]
            full_row = context + ", " + ", ".join(other_parts)

            # Break into multiple parts if necessary
            if len(full_row) <= max_len:
                return [full_row]
            else:
                segments = []
                current = context + ", "
                for part in other_parts:
                    if len(current) + len(part) + 2 > max_len:  # +2 for ", "
                        segments.append(current.rstrip(", "))
                        current = context + ", " + part + ", "
                    else:
                        current += part + ", "
                if current:
                    segments.append(current.rstrip(", "))
                return segments

        chunks = []
        current_chunk = ""

        for _, row in df.iterrows():
            segments = row_to_text_segments(row, MAX_CHARS)

            for segment in segments:
                segment += "\n\n"
                if len(current_chunk) + len(segment) <= MAX_CHARS:
                    current_chunk += segment
                else:
                    chunks.append(current_chunk.strip())
                    current_chunk = segment
        if current_chunk:
            chunks.append(current_chunk.strip())

        # Join and save
        final_text = '\n\n'.join(chunks)
        base_name = os.path.splitext(filename)[0].replace(' ', '_')
        output_file = os.path.join(output_folder, f"{base_name}.txt")

        with open(output_file, 'w') as f:
            f.write(final_text)

        print(f"Saved {output_file} with {len(chunks)} chunks.")


Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/skaters_(9).txt with 22175 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/goalies.txt with 445 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/goalies_(4).txt with 410 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/lines_(5).txt with 50304 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/skaters_(15).txt with 23775 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/skaters_(5).txt with 20975 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/teams_(2).txt with 600 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/goalies_(14).txt with 535 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbi

In [30]:
folder_path = '/Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data'
csv_folder = os.path.join(folder_path, 'csvs')
# output_folder = os.path.join(folder_path, 'texts')
file_path = os.path.join(csv_folder, "skaters (9).csv")
df = pd.read_csv(file_path)

In [31]:
for x in df.columns:
    print(x)
print(len(df.columns))

playerId
season
name
team
position
situation
games_played
icetime
shifts
gameScore
onIce_xGoalsPercentage
offIce_xGoalsPercentage
onIce_corsiPercentage
offIce_corsiPercentage
onIce_fenwickPercentage
offIce_fenwickPercentage
iceTimeRank
I_F_xOnGoal
I_F_xGoals
I_F_xRebounds
I_F_xFreeze
I_F_xPlayStopped
I_F_xPlayContinuedInZone
I_F_xPlayContinuedOutsideZone
I_F_flurryAdjustedxGoals
I_F_scoreVenueAdjustedxGoals
I_F_flurryScoreVenueAdjustedxGoals
I_F_primaryAssists
I_F_secondaryAssists
I_F_shotsOnGoal
I_F_missedShots
I_F_blockedShotAttempts
I_F_shotAttempts
I_F_points
I_F_goals
I_F_rebounds
I_F_reboundGoals
I_F_freeze
I_F_playStopped
I_F_playContinuedInZone
I_F_playContinuedOutsideZone
I_F_savedShotsOnGoal
I_F_savedUnblockedShotAttempts
penalties
I_F_penalityMinutes
I_F_faceOffsWon
I_F_hits
I_F_takeaways
I_F_giveaways
I_F_lowDangerShots
I_F_mediumDangerShots
I_F_highDangerShots
I_F_lowDangerxGoals
I_F_mediumDangerxGoals
I_F_highDangerxGoals
I_F_lowDangerGoals
I_F_mediumDangerGoals
I_F_highD